### * 安裝套件包

In [ ]:
# import need packages
import os
# 爬蟲用
import requests 
from bs4 import BeautifulSoup
import urllib3
# 處理資料用
import re
import time
import random
import datetime
# linebot-sdk packages
from flask import Flask, request, abort
from linebot import LineBotApi, WebhookHandler
from linebot.exceptions import InvalidSignatureError
from linebot.models import *

### * 設定與 LINE API TOKEN
##### - 參考連結：https://github.com/line/line-bot-sdk-python
##### - 可直接複製 <@Synopsis> 區塊部分

In [ ]:
# 基本設定
app = Flask(__name__)

# Line Messenger API for your channels
Channel_Access_Token = "請將你的Line_develop TOKEN填入 "
line_bot_api = LineBotApi(Channel_Access_Token)

# Channel Secret
Channel_Secret = "請將你的Line_develop Secret填入 "
handler = WebhookHandler(Channel_Secret)

# Imgur 圖片網站 api
# 若需要圖片 https://github.com/Imgur/imgurpython可以透過 api獲取及上傳image
client_id = "請將你的client_id填入"
client_secret = "請將你的client_secret填入"


# requests.get 偽裝器
# 在使用requests時，讓瀏覽器辨識你為一般使用者 (有些網站會設立防機器人裝置)
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.63 Safari/537.36"}

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    # print("body:",body)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'ok'

### * 開始建立 你的專屬-Line bot功能

#### - 我主要設立的目的是爬取ptt上的文章，並將「本日爆文」整理給用戶
#### - 以及一些新聞網站的爬蟲並彙整資料，提供給用戶
#### - 是一個專屬 懶惰鄉民用的工具 
#### - BTW 網上有蠻多爬蟲教學示範，各位有興趣可以自行上網學習

In [ ]:
# 一些需要用的 functions
# 處理 title 文字(稍後處理資料會用到)
def del_re(string):
    import re
    if re.search(r'^Re.\s*',string) != None:
        count = re.search(r'^Re.\s*',string).span()[1]
        return string[count:]
    else:
        return string
# 今日時間
def today_date():
    today = datetime.date.today()
    today_format = today.strftime(("%m/%d"))[1:]  # 與ptt網站時間格式相同
    return today_format
# 昨日時間
def yesterday_date():
    yesterday = datetime.date.today() + datetime.timedelta(-1)
    yesterday_format = yesterday.strftime("%m/%d")[1:] # 與ptt網站時間格式相同
    return yesterday_format

# 抓取上一頁的按鈕 url
def upper_url(url,times=5):  # 
    url_list = [url]
    for i in range(times):
        res = requests.get(url)
        soup = BeautifulSoup(res.text,"html.parser")
        up_page_href = soup.select('div.btn-group a')[3]['href']
        up_page_url = 'https://www.ptt.cc' + up_page_href
        url_list.append(up_page_url)
        url = up_page_url
    return url_list

### * _main__主程式

##### @ ptt文章

In [ ]:
# __main__主程式
# ptt_nba版 文章
# 今日nba
def ptt_nba_today():
    articles = []
    curr_url = 'https://www.ptt.cc/bbs/NBA/index.html' # 爬蟲網址，以ptt nba為例
    url_list = upper_url(curr_url)
    for url in url_list:
        rs = requests.sessions.Session()
        res = rs.get(url,headers=headers)
        soup = BeautifulSoup(res.text,'html.parser')
        for r_ent in soup.select('div.r-ent'):
            try:
                href_ = r_ent.select_one('a')
                if href_ != None:
                    link = 'https://www.ptt.cc' + href_['href']    # 每篇文章的連結
                    title = href_.text.strip()                     # 每篇文章的標題
                    rate = r_ent.select_one('div.nrec').text       # 每篇文章的留言數
                    date = r_ent.select_one('.date').text.strip()  # 每篇文章的日期
                    if rate != None:
                        if rate.isnumeric() and int(rate) >= 50:
                            rate = '推{}'.format(rate)
                            articles.append({'date':date ,'rate':rate,'title':del_re(title) ,'link':link})
                        else:
                            if rate.startswith('X'):
                                rate = '噓🤫'
                                articles.append({'date':date ,'rate':rate ,'title':del_re(title) ,'link':link})
                            if rate == '爆':
                                rate = '爆🔥'
                                articles.append({'date':date ,'rate':rate ,'title':del_re(title) ,'link':link})
            except Exception as e:
                print('文章已被刪除', e)

    content_today  = '<0{}>\n{}\n\n'.format(today_date(),'='*7)
    for index ,article in enumerate(articles,1):
        if article.get('date') == today_date():
            if index == 20 :
                return content_today
            data = '{} {}\n{}\n\n'.format(article.get('rate'),article.get('title'), article.get('link'))
            content_today += data
    return content_today

# 昨日Nba
def ptt_nba_yest():
    articles = []
    curr_url = 'https://www.ptt.cc/bbs/NBA/index.html'
    url_list = upper_url(curr_url)
    for url in url_list:
        rs = requests.sessions.Session()
        res = rs.get(url,headers=headers)
        soup = BeautifulSoup(res.text,'html.parser')
        for r_ent in soup.select('div.r-ent'):
            try:
                href_ = r_ent.select_one('a')
                if href_ != None:
                    link = 'https://www.ptt.cc' + href_['href']    # 每篇文章的連結
                    title = href_.text.strip()                     # 每篇文章的標題
                    rate = r_ent.select_one('div.nrec').text       # 每篇文章的留言數
                    date = r_ent.select_one('.date').text.strip()  # 每篇文章的日期
                    if rate != None:
                        if rate.isnumeric() and int(rate) >= 50:
                            rate = '推{}'.format(rate)
                            articles.append({'date':date ,'rate':rate,'title':del_re(title) ,'link':link})
                        else:
                            if rate.startswith('X'):
                                rate = '噓🤫'
                                articles.append({'date':date ,'rate':rate ,'title':del_re(title) ,'link':link})
                            if rate == '爆':
                                rate = '爆🔥'
                                articles.append({'date':date ,'rate':rate ,'title':del_re(title) ,'link':link})
            except Exception as e:
                print('文章已被刪除', e)

    content_yest  = '<0{}>\n{}\n\n'.format(yesterday_date(),'='*7)
    for index ,article in enumerate(articles,1):
        if article.get('date') == yesterday_date():
            if article.get('rate') == '爆🔥':
                data = '{} {}\n{}\n\n'.format(article.get('rate'),article.get('title'), article.get('link'))
                content_yest += data
    return content_yest

In [ ]:
# Sex 西斯版 文章
def ptt_sex():
    urllib3.disable_warnings()
    rs = requests.sessions.Session()
    # 需跳轉 18 禁登入頁面
    load = {
        'from': '/bbs/sex/index.html',
        'yes': 'yes'
    }
    res = rs.post('https://www.ptt.cc/ask/over18', verify=False, data=load)
    soup = BeautifulSoup(res.text,'html.parser')
    article_dict = []
    for r_ent in soup.select('div.r-ent'):
        try:
            href_ = r_ent.select_one('a')
            if href_ != None:
                title = href_.text.strip()
                rate = r_ent.select_one('div.nrec').text
                url = 'https://www.ptt.cc' + href_['href']
                if rate != None:
                    if rate.isnumeric():
                        rate = '推 {}+'.format(rate)
                    elif rate.startswith('X'):
                        rate = '噓'
                    elif rate == '':
                        rate = '推 0+'
                    else:
                        rate = '🈲'+rate
                    article_dict.append({'title':title,'url':url,'rate':rate})
        except Exception as e:
            print('本文已被刪除', e)
    content = ''
    for index, article in enumerate(article_dict, 0):
        # 最後五篇為置頂五篇，所以只要取到15後，置頂就不用取
        if index == 15:
            return content
        data = '[{}] {}\n{}\n\n'.format(article.get('rate'),article.get('title'), article.get('url'))
        content += data
    return content

##### @ 新聞時事彙整

In [ ]:
# 狂新聞
def news_crazy_new():
    urllib3.disable_warnings()
    target_url = 'https://ck101.com/'
    rs = requests.session()
    res = rs.get(target_url, headers=headers, verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')
    articles = []
    upper = soup.select('div.newPosts__upper h4 a')
    for unews in upper:
        title = unews.text.strip()
        link = unews.get('href')
        articles.append({'title': title, 'link': link})
    time.sleep(1)
    bottom = soup.select('div.newPosts__bottom h4 a')
    for bnews in bottom:
        title = bnews.text.strip()
        link = bnews.get('href')
        articles.append({'title': title, 'link': link})

    random.shuffle(articles)
    content = '<卡提諾狂新聞>\n{}\n'.format('========')
    for index, article in enumerate(articles, 1):
        if index == 6:
            return content
        content += '[狂] {}\n{}\n\n'.format(article.get('title'), article.get('link'))

    return content

# 最新狂影片
def news_crazy_video():
    urllib3.disable_warnings()
    target_url = "https://ck101.com/forum.php?mod=forumdisplay&fid=3731"
    rs = requests.session()
    res = rs.get(target_url ,headers=headers ,verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')

    time.sleep(1)
    articles = []
    for video in soup.select('h3 a'):
        string = video.get('onclick')
        if string != None:
            title = video.get('title').strip()
            link = re.findall(r'http://ck101.com/.*.html',string).pop()
            articles.append({'title':title,'link':link})

    content = '<卡提諾狂影集>\n{}\n'.format('========')
    for index ,article in enumerate(articles,1):
        if index == 5:
            return content
        content += '[狂] {}\n{}\n\n'.format(article.get('title'), article.get('link'))

    return content

# 狂惡搞 kuso
def news_crazy_kuso():
    urllib3.disable_warnings()
    target_url = 'https://ck101.com/forum.php?mod=forumdisplay&fid=3607&page=1'
    rs = requests.session()
    res = rs.get(target_url, headers=headers, verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')

    time.sleep(1)
    articles = []
    for kuso in soup.select('div.blockTitle a'):
        string = kuso.get('onclick')
        if string != None:
            title = kuso.get('title').strip()
            link = re.findall(r'http://ck101.com/.*.html', string).pop()
            articles.append({'title': title, 'link': link})

    # 第一則為版規
    articles = articles[1:]
    random.shuffle(articles)
    content = '<卡提諾狂Kuso>\n{}\n'.format('========')
    for index, article in enumerate(articles, 1):
        if index == 8:
            return content
        content += '[狂] {}\n{}\n\n'.format(article.get('title'), article.get('link'))

    return content

In [ ]:
# 科技新聞 tech
# 科技新報
def technews():
    target_url = 'https://technews.tw/'
    print('Start parsing movie ...')
    rs = requests.session()
    res = rs.get(target_url, verify=False)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'html.parser')

    content = ""
    for index, data in enumerate(soup.select('article div h1.entry-title a')):
        if index == 12:
            return content
        title = data.text
        link = data['href']
        content += '{}\n{}\n\n'.format(title, link)
    return content
# 泛科技
def panx():
    target_url = "https://panx.asia/"
    print('Start parsing ptt hot....')
    rs = requests.session()
    res = rs.get(target_url, verify=False)
    soup = BeautifulSoup(res.text, 'html.parser')

    content = ""
    for data in soup.select('div.container div.row div.desc_wrap h2 a'):
        title = data.text
        link = data['href']
        content += '{}\n{}\n\n'.format(title, link)
    return content

### * LINEBOT 回覆設定

In [ ]:
# 腳本設定： 如果 user 訊息為{＿＿}就執行哪個程式，並回傳到 MessageEvent
# ================================================
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    print("event.reply_token:", event.reply_token)
    print("event.message.text:", event.message.text)

    # NBA文章(單一訊息處理，call 呼叫程式)
    # ============================================
    if event.message.text in ['Nba today','nba today','NBA today']:
        content = ptt_nba_today()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0

    if event.message.text in ['Nba yest','nba yest','NBA yest']:
        content = ptt_nba_yest()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0

    # Sex 文章
    # ============================================
    if event.message.text in ['sex','Sex','SEX','西斯']:
        content = ptt_sex()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0

    # 新聞News
    # ============================================
    if event.message.text == "狂新聞":
        content = news_crazy_new()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0

    if event.message.text == "狂影片":
        content = news_crazy_video()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0

    if event.message.text == "狂kuso":
        content = news_crazy_kuso()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0

    if event.message.text == "PanX泛科技":
        content = panx()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0

    if event.message.text == "科技新報":
        content = technews()
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=content))
        return 0

In [ ]:
# 圖文訊息處理
# ============================================
    if event.message.text == '開始':
        buttons_template = TemplateSendMessage(
            alt_text='Buttons template',
            template=ButtonsTemplate(
                thumbnail_image_url='必須是https圖檔的url'
                title='我是鄉民30CM',
                text='請選擇內容',
                actions=[
                    MessageTemplateAction(
                        label='來當個鄉民',
                        text='ptt'
                    ),
                    MessageTemplateAction(
                        label='來看個新聞',
                        text='新聞'
                    ),
                    MessageTemplateAction(
                        label='來點正妹圖',
                        text='正妹圖'
                    )
                ]
            )
        )
        line_bot_api.reply_message(event.reply_token, buttons_template)
        return 0

    if event.message.text == '新聞':
        buttons_template = TemplateSendMessage(
            alt_text='Buttons template',
            template=ButtonsTemplate(
                thumbnail_image_url='https://365psd.com/images/previews/1d0/psd-news-icon-53281.jpg',
                title='新聞進行式',
                text='請選擇內容',
                actions=[
                    MessageTemplateAction(
                        label='Google新聞',
                        text='GoogleNews'
                    ),
                    MessageTemplateAction(
                        label='卡提諾狂新聞',
                        text='卡提諾'
                    ),
                    MessageTemplateAction(
                        label='蘋果動新聞',
                        text='蘋果新聞'
                    ),
                    MessageTemplateAction(
                        '科技新聞',
                        text='科技新聞'
                    )
                ]
            )
        )
        line_bot_api.reply_message(event.reply_token, buttons_template)
        return 0

    if event.message.text == '卡提諾':
        buttons_template = TemplateSendMessage(
            alt_text='Buttons template',
            template=ButtonsTemplate(
                thumbnail_image_url='https://img.ltn.com.tw/Upload/liveNews/BigPic/600_phpg6IvSl.jpg',
                title='卡提諾狂新聞',
                text='請選擇內容',
                actions=[
                    MessageTemplateAction(
                        label='熱門狂新聞',
                        text='狂新聞'
                    ),
                    MessageTemplateAction(
                        label='卡提諾狂影集',
                        text='狂影片'
                    ),
                    MessageTemplateAction(
                        label='卡提諾KUSO',
                        text='狂kuso'
                    )
                ]
            )
        )
        line_bot_api.reply_message(event.reply_token, buttons_template)
        return 0

    if event.message.text == '科技新聞':
        buttons_template = TemplateSendMessage(
            alt_text='Buttons template',
            template=ButtonsTemplate(
                thumbnail_image_url='https://geospatialmedia.s3.amazonaws.com/wp-content/uploads/2019/02/Techcircle.jpg',
                title='Technology News',
                text='請選擇內容',
                actions=[
                    MessageTemplateAction(
                        label='PanX 泛科技',
                        text='PanX泛科技'
                    ),
                    MessageTemplateAction(
                        label='TechNews',
                        text='科技新報'
                    )
                ]
            )
        )
        line_bot_api.reply_message(event.reply_token, buttons_template)
        return 0

    if event.message.text == 'ptt':
        buttons_template = TemplateSendMessage(
            alt_text='Buttons template',
            template=ButtonsTemplate(
                thumbnail_image_url='https://photo.sofun.tw/2015/06/Aotter-PTT-Logo.png',
                title='Menu',
                text='請選擇任一版',
                actions=[
                    MessageTemplateAction(
                        label='NBA(籃球版)',
                        text='nba today'
                    ),
                    MessageTemplateAction(
                        label='Sex(西斯版)',
                        text='sex'
                    ),
                    MessageTemplateAction(
                        label='HatesPolitic(政黑版)',
                        text='hater'
                    ),
                    MessageTemplateAction(
                        label='C_Chat(閒談版)',
                        text='chat'
                    )
                ]
            )
        )
        line_bot_api.reply_message(event.reply_token, buttons_template)
        return 0

### ＊ 將撰寫好的code上傳到伺服器

In [ ]:
# 最後上傳 Heroku
if __name__ == "__main__":
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port)

### ＊完成 DONE 